Global setup

In [20]:
n=4
k=2
q=41 #Alkaline D
#q should be divisible by 8 for NTT
eta=1 #secret key range, prefered 2 in Alkaline
gamma1=2^4 #y coeff range, power of 2, exponent about 2/3 of q?  Earlier version was *not* power of 2
gamma2=(q-1)/4 #low-order rounding range, large(?) factor of q-1
R.<x> = ZZ[]

In [21]:
q

41

In [22]:
is_prime(q)

True

In [23]:
gamma2

10

In [24]:
for tau in range(5):
    print(ln(binomial(4, tau)).n()+tau) #challenge entropy, should be between n/2 and n

0.000000000000000
2.38629436111989
3.79175946922805
4.38629436111989
4.00000000000000


In [25]:
tau = 1 #number of 1's in c
ln(binomial(4, tau)).n()+tau

2.38629436111989

In [26]:
beta = tau*eta
beta

1

Chance of $\mathbf{y}$-coefficient passing:

In [27]:
((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k).n()

0.586530272919658

Chance of low-order rounding passing:

In [28]:
((2*(gamma2-beta)-1)/(2*gamma2-1))^(n*k).n()

0.410735805211182

Expected number of repetitions:

In [29]:
1/(((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k))*1/(((2*(gamma2-beta)-1)/(2*gamma2-1))^(n*k)).n()

4.15094523954013

Alice generates a signing key

In [30]:
#A = matrix([[randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)], 
#            [randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)]])
A = matrix([[ 2*x^3 + 36*x^2 + 4*x + 10, 3*x^3 + 15*x^2 + 29*x + 1],
[21*x^3 + 4*x^2 + 32*x + 18, 10*x^3 + 11*x^2 + 34*x + 1]])
A

[ 2*x^3 + 36*x^2 + 4*x + 10  3*x^3 + 15*x^2 + 29*x + 1]
[21*x^3 + 4*x^2 + 32*x + 18 10*x^3 + 11*x^2 + 34*x + 1]

In [31]:
#s1 = matrix([[randrange(-eta,eta)*x^3+randrange(-eta,eta)*x^2+randrange(-eta,eta)*x+randrange(-eta,eta)], 
#            [randrange(-eta,eta)*x^3+randrange(-eta,eta)*x^2+randrange(-eta,eta)*x+randrange(-eta,eta)]])
s1 = matrix([[-x^3 - x^2 - x - 1],
[-1]])
s1

[-x^3 - x^2 - x - 1]
[                -1]

In [32]:
#s2 = matrix([[randrange(-eta,eta)*x^3+randrange(-eta,eta)*x^2+randrange(-eta,eta)*x+randrange(-eta,eta)], 
#            [randrange(-eta,eta)*x^3+randrange(-eta,eta)*x^2+randrange(-eta,eta)*x+randrange(-eta,eta)]])
s2 = matrix([[-x^3 - x],
[-x^3 - x - 1]])
s2

[    -x^3 - x]
[-x^3 - x - 1]

$s_1$ and $s_2$ are private

In [33]:
t = matrix(((A*s1 + s2) % (x^n+1) % q))
t

[ 26*x^3 + 19*x^2 - 6*x + 31]
[37*x^3 + 38*x^2 + 22*x + 37]

$A$ and $t$ are public

Alice signs the message

In [34]:
M = 'hi!'
M

'hi!'

In [35]:
#y = matrix([[randrange(-(gamma1-1), gamma1-1)*x^3+randrange(-(gamma1-1), gamma1-1)*x^2+randrange(-(gamma1-1), gamma1-1)*x+randrange(-(gamma1-1), gamma1-1)], 
#            [randrange(-(gamma1-1), gamma1-1)*x^3+randrange(-(gamma1-1), gamma1-1)*x^2+randrange(-(gamma1-1), gamma1-1)*x+randrange(-(gamma1-1), gamma1-1)]])
y = matrix([[14*x^3 - 12*x^2 - 11*x + 11],
[10*x^3 - 7*x^2 - 10*x + 11]])
y

[14*x^3 - 12*x^2 - 11*x + 11]
[ 10*x^3 - 7*x^2 - 10*x + 11]

In [36]:
def high_bits(r, alpha):
    r = r % q
    r0 = Mod(r, alpha).lift_centered()
    if r-r0 == q-1:
        return 0
    else:
        return (r-r0)/alpha

def low_bits(r, alpha):
    r = r % q
    r0 = Mod(r, alpha).lift_centered()
    if r-r0 == q-1:
        return r0-1
    else:
        return r0

In [37]:
matrix(((A*y) % (x^n+1) % q))

[23*x^3 + x^2 + 3*x + 36]
[  32*x^3 + 4*x^2 + 18*x]

In [38]:
w1 = matrix(((A*y) % (x^n+1) % q)).apply_map(lambda f: f.map_coefficients(lambda r: high_bits(r, 2*gamma2) ))
w1

[x^3]
[  x]

In [39]:
from sage.cpython.string import str_to_bytes

str_to_bytes(M+str(w1))

b'hi![x^3]\n[  x]'

In [40]:
import hashlib

c_hash = hashlib.shake_256(str_to_bytes(M+str(w1)))

In [41]:
def sample_in_ball(digest):  # digest should be twice the length of c
    c = [0] * 4
    for i in range(4-tau, 4):
        j = digest[i] % (i+1)
        s = digest[i+4] % 2
        c[i] = c[j]
        c[j] = (-1)^s
    return c
    

In [42]:
c = R(sample_in_ball(c_hash.digest(8)))
c

x^3

In [43]:
z = matrix(((y + c*s1) % (x^n+1) % q))
z

[13*x^3 - 11*x^2 - 10*x + 12]
[  9*x^3 - 7*x^2 - 10*x + 11]

Is $\|\mathbf{z}\|_\infty \geq \gamma_1-\beta$?

In [44]:
gamma1-beta

15

If so, reject and pick a new $\mathbf{y}$|

Is $\|\mathrm{LowBits}(\mathbf{Ay}-c\mathbf{s}_2, 2\gamma_2)\|_\infty \geq \gamma_2-\beta$?

In [45]:
matrix(((A*y - c*s2) % (x^n+1) % q)).apply_map(lambda f: f.map_coefficients(lambda r: low_bits(r, 2*gamma2) ))

[         3*x^3 + 3*x - 6]
[-8*x^3 + 3*x^2 - 2*x - 1]

In [46]:
gamma2-beta

9

If so, reject and pick a new $\mathbf{y}$

If not, $(\mathbf{z}, c)$ is the signature

In [47]:
(z, c)

(
[13*x^3 - 11*x^2 - 10*x + 12]     
[  9*x^3 - 7*x^2 - 10*x + 11], x^3
)

Bob verifies the signature

In [48]:
w1prime = matrix(((A*z-c*t) % (x^n+1) % q)).apply_map(lambda f: f.map_coefficients(lambda r: high_bits(r, 2*gamma2) ))
w1prime

[x^3]
[  x]

Is $\|\mathbf{z}\|_\infty < \gamma_1-\beta$?

In [49]:
z

[13*x^3 - 11*x^2 - 10*x + 12]
[  9*x^3 - 7*x^2 - 10*x + 11]

In [50]:
gamma1-beta

15

Yes

Is $c = H(M \| w_1')$?

In [51]:
cprime_hash = hashlib.shake_256(str_to_bytes(M+str(w1prime)))

In [52]:
cprime = R(sample_in_ball(cprime_hash.digest(8)))
cprime

x^3

In [53]:
c == cprime

True

Yes.  The signature is accepted.